In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

from datetime import date

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

import configparser
config = configparser.ConfigParser()
config.read('config.ini')

import wandb
wandb.login()

2.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


wandb: Currently logged in as: mhrnciar (nsiete-hrnciar-katkovcin). Use `wandb login --relogin` to force relogin


True

In [ ]:
name_of_run = 'first-test'
run = wandb.init(project="gan-portraits", id=name_of_run)